In [10]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import QuantLib as ql

from functions import *

In [11]:
def price_swap(settlement_date, 
               maturity_date, 
               discount_curve, 
               calendar, 
               tenor_fixed_leg,
               tenor_floating_leg, 
               swap_rate,
               current_coupon_rate, 
               spread, 
               notional, 
               day_count_fixed_leg,
               day_count_floating_leg, 
               day_rolling, 
               schedule_fixed_leg,
               schedule_floating_leg,
               is_receiver = True):

    fixed_leg = price_coupon_bond(
        settlement_date,
        maturity_date,
        discount_curve,
        calendar,
        tenor_fixed_leg,
        swap_rate,
        notional,
        day_count_fixed_leg,
        day_rolling,
        schedule_fixed_leg        
    )

    floating_leg = price_floating_rate_note(
        settlement_date,
        maturity_date,
        discount_curve,
        calendar,
        tenor_floating_leg,
        current_coupon_rate,
        spread,
        notional,
        day_count_floating_leg,
        day_rolling,
        schedule_floating_leg
    )

    fixed_leg_discounted_cash_flows = fixed_leg["Summary Table"]["Discounted Cash Flows"].values
    floating_leg_discounted_cash_flows = floating_leg["Summary Table"]["Discounted Cash Flows"].values

    price = np.sum(floating_leg_discounted_cash_flows)-np.sum(fixed_leg_discounted_cash_flows)

    if is_receiver == False:
            price = -price

    return {"Price": price, 
            "Summary Table Fixed Leg": fixed_leg["Summary Table"], 
            "Summary Table Floating Leg": floating_leg["Summary Table"]}

In [12]:
notional = 100000

calendar = ql.TARGET()
settlement_date = ql.Date(12,11,2025)
maturity_date = ql.Date(10,10,2028)

day_rolling = ql.ModifiedFollowing
termination_day_rolling = ql.ModifiedFollowing
end_of_month = False
date_generation = ql.DateGeneration.Backward

discount_curve = pd.read_excel("tsData2025.xlsx", index_col= 0)

# Fixed Leg
swap_rate = 0.02
tenor_fixed = ql.Period(1,ql.Years)
day_count_fixed = ql.ActualActual(ql.ActualActual.ISDA)

schedule_fixed = ql.Schedule( settlement_date,
                        maturity_date,
                        tenor_fixed,
                        calendar,
                        day_rolling,
                        termination_day_rolling,
                        date_generation,
                        end_of_month) 

# Floating Leg
current_coupon = 0.0340
spread = 0
tenor_floating = ql.Period(6,ql.Months)
day_count_floating = ql.Actual360()

schedule_floating = ql.Schedule( settlement_date,
                        maturity_date,
                        tenor_floating,
                        calendar,
                        day_rolling,
                        termination_day_rolling,
                        date_generation,
                        end_of_month)  

In [13]:
swap = price_swap(
    settlement_date,
    maturity_date,
    discount_curve,
    calendar,
    tenor_fixed,
    tenor_floating,
    swap_rate,
    current_coupon,
    spread,
    notional,
    day_count_fixed,
    day_count_floating,
    day_rolling,
    schedule_fixed,
    schedule_floating,
    is_receiver= False
)

In [14]:
swap["Price"]

2764.895218759644